__Obiettivo__

Estrarre alcuni metadati relativi ai file PDF contenuti nella cartella _articles_, mediante l'impiego di apposite librerie per la gestione e manipolazione di file PDF.

I dati estratti sono suddivisi in:
- __DOI__, Digital Object Identifier, identificativo univoco di risorse digitali
- __Title__, titolo dell'articolo / paper scientifico 
- __Author__, autore / autori partecipanti alla stesura dell'articolo preso in considerazione
- __Abstract__, riassunto di un documento, privo di interpretazioni o critiche

_DOI_ è l'acronimo di __Digital Object Identifier__, identificativo univoco di risorse digitali. Per poter apprendere tale informazione è necessario utilizzare la _REST API_ fornita da __CrossRef__; CrossRef è un'infrastuttura digitale dedita alle memorizzazione di tutti gli articoli posti in ambito accademico. Di seguito, si percepisce l'importanza del titolo e dell'autore per ogni PDF, affinchè interrogando la _Application Programming Interface_ sia possibile ottenere il DOI.

In [ ]:
target_word = "abstract"
pdf_path = "../articles/"
crossref_url = "https://api.crossref.org/works/10.1037/0003-066X.59.1.29/agency"

Le tre classi riportate sono utilizzate rispettivamente per:
- __ScannedText__, contiene sezioni del file PDF scannerizzato, dove _index_ rappresenta l'indice della linea in cui compare la _key word_ ricercata, mentre _text_ è una variabile _str_ utilizzata per memorizzare la scansione da immagine a stringa effettuata tramite _pytesseract_
- __Metadata__, ciascun oggetto istanziato della classe rappresenta i metadati ottenuti di ogni singolo file presente all'interno della cartella _articles_
- __AgentExtractor__, implementata per realizzare una __chain__ secondo le direttive della libreria __langchain__. Utilizzata per poter estrarre il _titolo_ e gli _autori_ degli articoli privi di _metadati_ già presenti

In [187]:
class ScannedText:
    def __init__(self, index, text):
        self.index = index
        self.text = text

In [188]:
from typing import Dict

class Metadata:
    def __init__(self, DOI=None, path=None, title=None, author=None, abstract=None):
        self.DOI = DOI
        self.path = path
        self.title = title
        self.author = author
        self.abstract = abstract

    def get_dict(self) -> Dict[str, Dict[str, str]]:
        return {
            self.path: {
                "DOI": self.DOI,
                "Title": self.title,
                "Author": self.author,
                "Abstract": self.abstract
            }
        }

In [ ]:
import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

load_dotenv()

class AgentExtractor:
    def __init__(self):
        system_message = """
            You are an assistant in charge to extract the title and the authors' list of scientific paper.

            You must extract at least the title and the authors, any other information is not requested. Return all the
            fields required like an unique string; all the fields must be separated by a comma.

            Example of an extraction:
            Title, First Author, Second Author, ..., Last Author
        """

        prompt = ChatPromptTemplate(
            [
                ("system", system_message),
                ("human", "Scientific paper, which will be used to extract the title and the authors' list. \n {text}")
            ]
        )

        llm = ChatOpenAI(
            model="gpt-4o",
            api_key=os.getenv("OPENAI_KEY"),
            temperature=0
        )

        self.agent = prompt | llm

    def get_agent(self):
        return self.agent

La pipeline è composta da due fasi principali:
- _Estrazione_ dei metadati
- _Elaborazione_ dei metadati ottenuti

Le librerie utilizzate per l'acquisizione dei metadati si fondano a loro volta sulla libreria __pdfMiner__, pertanto riescono ad estrapolare le informazioni circoscritte dagli stessi file. Perciò tutte le librerie trovate si equivalgono.

Tramite la funzione _filter_, associata al metodo _query_ fornita dalla libreria __crossref__, è possibile recuperare l'_abstract_ degli articoli presenti all'interno dell'API. Tuttavia, è stato notato che l'impiego del filtro comporta ad una risposta completamente differente rispetto alla casistica in cui sia assente. Di seguito, è presentato lo snippet di codice implementato:

<div align="center">
    <p><b>url_request = work.query(bibliographic=title, author=author).filter(has_abstract=true).url</b></p>
</div>

In [190]:
from typing import List

def get_path_pdf_files(pdf_path: str) -> List[str]:
    path_files = []

    for path in os.listdir(pdf_path):
        path_files.append(pdf_path + path)
    
    return path_files

path_pdf_files = get_path_pdf_files(pdf_path)

In [191]:
import pytesseract

from pdf2image import convert_from_path

def convert_file_to_images(path: str) -> list:
    try:
        return convert_from_path(path)[:3]
    except Exception as e:
        print("Error during conversion from file to image:", e)

def scan_file_to_text(paths: list[str]) -> dict[str, str]:
    dict_texts: Dict[str, str] = {}

    for path in paths:
        images = convert_file_to_images(path)

        text = ""
        try:
            for image in images:
                text = text + pytesseract.image_to_string(image)

            dict_texts[path] = text
        except Exception as e:
            print("Error during conversion from image to text:", e) 
        
    return dict_texts

scanned_text = scan_file_to_text(path_pdf_files)

def get_target_word_index(keyword: str, text: str) -> int:
    count_line = 0

    lines = text.splitlines()
    for line in lines:
        if keyword in line:
            return count_line
        
        count_line += 1
    
    return -1

def detect_target_line(target_word:str, dict: dict[str, str]) -> dict[str, ScannedText]:
    dict_lines: Dict[str, ScannedText] = {}

    for key, value in dict.items():
        dict_lines[key] = ScannedText(get_target_word_index(target_word, value.lower()), value)

    return dict_lines

abstracts = detect_target_line("abstract", scanned_text)
introductions = detect_target_line("introduction", scanned_text)

In [192]:
import json
import pymupdf
import pdfplumber

from pypdf import PdfReader

def get_title_from_dicts(pdfreader: str, pymupdf: dict[str, str], pdfplumber: dict[str, str]) -> str:
    return (pdfreader or pymupdf.get("title") or pdfplumber.get("title") or "Not found")

def get_author_from_dicts(pdfreader: str, pymupdf: dict[str, str], pdfplumber: dict[str, str]) -> str:
    return (pdfreader or pymupdf.get("author") or pdfplumber.get("author") or "Not found")

def extract_title_and_author(i: int, len: int, paths: list[str], list_metadata: List[Metadata] = []) -> List[Metadata]:
    if len == 0:
        return list_metadata
    else:
        _pdfreader = PdfReader(paths[i]).metadata
        _pymupdf = pymupdf.open(paths[i]).metadata
        _pdfplumber = pdfplumber.open(paths[i]).metadata

        title = get_title_from_dicts(_pdfreader.title, _pymupdf, _pdfplumber)
        author = get_author_from_dicts(_pdfreader.author, _pymupdf, _pdfplumber)

        list_metadata.append(Metadata(path=paths[i], title=title, author=author))

        extract_title_and_author(i + 1, len - 1, paths)
        return list_metadata

list_metadata = extract_title_and_author(0, len(path_pdf_files), path_pdf_files)

[WARNING] Ignoring wrong pointing object 2 65536 (offset 0)
[WARNING] Ignoring wrong pointing object 16 65536 (offset 0)
[WARNING] Ignoring wrong pointing object 49 65536 (offset 0)
[WARNING] Ignoring wrong pointing object 59 65536 (offset 0)
[WARNING] Ignoring wrong pointing object 63 65536 (offset 0)
[WARNING] Ignoring wrong pointing object 72 65536 (offset 0)
[WARNING] Ignoring wrong pointing object 76 65536 (offset 0)
[WARNING] Ignoring wrong pointing object 86 65536 (offset 0)


Piccolo catalogo di articoli a cui manca anche il titolo e autore:
- 09CiancariniFavini
- 16DavidNetanyahuWolf
- 19Kamlish
- ICGA_J_34_2_HHB_Zugzwangs_in_Chess_Studies

In [193]:
def get_authors_name(i: int, len:int, authors: list[str], field = "") -> str:
    if len == 1:
        return field + authors[i]
    else:
        return get_authors_name(i + 1, len - 1, authors, field + authors[i] + " and ")
    
extractor = AgentExtractor()
agent_extractor = extractor.get_agent()

for metadata in list_metadata:
    if "Not found" in (metadata.title, metadata.author):
        metadata.title = ""
        metadata.author = ""

        answer = agent_extractor.invoke(scanned_text.get(metadata.path)).content
        
        fields = answer.split(", ")

        metadata.title = fields[0]
        metadata.author = get_authors_name(0, len(fields[1:]), fields[1:])

[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [194]:
from typing import List

def remove_white_spaces(index: int, text: str) -> List[str]:
    lines = text.splitlines()
    first_part = lines[:index + 1]

    start_index = index + 1
    count_line = start_index

    for line in lines[start_index:]:
        if len(line) == 0:
            count_line += 1
            continue
        else:
            break
        
    second_part = lines[count_line:]
    
    return first_part + second_part

def extract_section_lines(index: int, lines: list[str]) -> str:
    abstract = ""

    for line in lines[index:]:
        if len(line) == 0:
            break

        abstract += line + "\n"

    return abstract

def extract_section(_dict: dict[str, ScannedText]) -> dict[str, str]:
    dict_section: Dict[str, str] = {}

    for key, value in _dict.items():

        if value.index > -1:
            abstract_lines = remove_white_spaces(value.index, value.text)
            text = extract_section_lines(value.index, abstract_lines)
        else:
            text = "Not found"

        dict_section[key] = text

    return dict_section

dict_abstracts = extract_section(abstracts)
dict_introductions = extract_section(introductions)

In [ ]:
import time
import requests

from difflib import SequenceMatcher
from crossref.restful import Works

def similar(title_extracted:str, title_crossref: str) -> float:
    return SequenceMatcher(None, title_extracted, title_crossref).ratio()

work = Works()

def send_crossref_request(title: str, author: str) -> str:
    url_request = work.query(bibliographic=title, author=author).url

    # Attardare di un certo delay come definito dalla libreria per ogni richiesta successiva
    time.sleep(2)
    print(url_request)

    try:
        request = requests.get(url_request)

        match request.status_code:
            case 200:
                response = request.json()

                message = response.get("message")
                items = message.get("items")

                for item in items:
                    if similar(title, item["title"][0]) > 0.5:
                        return item["DOI"]
                                                                
                    continue
            case 400:
                raise Exception("Error during request to CrossRef REST API: Bad Request")
            case _:
                raise Exception("Error during request to CrossRef REST API")
    except Exception as e:
        print("Error during request to CrossRef REST API:", e)

for item in list_metadata:
    if "Not found" not in (item.title, item.author):
        doi =  send_crossref_request(item.title, item.author)

        if doi is not None:
            item.DOI = doi

    item.abstract = dict_abstracts[item.path]
    
    continue

https://api.crossref.org/works?query.author=Eli+%28Omid%29+David+and+Nathan+S.+Netanyahu+and+Lior+Wolf&query.bibliographic=DeepChess%3A+End-to-End+Deep+Neural+Network+for+Automatic+Learning+in+Chess
https://api.crossref.org/works?query.author=Michael+George+and+Jonathan+Schaeffer&query.bibliographic=Chunking+for+Experience
https://api.crossref.org/works?query.author=R.+Feldmann+and+P.+Mysliwietz+and+B.+Monien&query.bibliographic=A+Fully+Distributed+Chess+Program
https://api.crossref.org/works?query.author=Joe+Condon+and+Ken+Thompson&query.bibliographic=Belle+Chess+Hardware
https://api.crossref.org/works?query.author=Don+Beal&query.bibliographic=Intelligent+Systems%2C+Artificial+and+Human
Error during request to CrossRef REST API: 'title'
https://api.crossref.org/works?query.author=Paolo+Ciancarini+and+Gian+Piero+Favini&query.bibliographic=Plagiarism+detection+in+game-playing+software
https://api.crossref.org/works?query.author=G.+Haworth+and+H.+M.+J.+F.+van+der+Heijden+and+E.+Bleicher&

In [196]:
list_json = []
for metadata in list_metadata:
    list_json.append(metadata.get_dict())

with open("../json/metadata.json", "a") as file:
    json.dump(list_json, file, indent=2)

L'utilizzo di una __chain__, secondo le regole implementative espresse dalla libreria __langchain__, ha permesso l'estrazione del _titolo_ e dell'_autore_ per tutti i file non in possesso dei _metadati_ ricercati. Tuttavia, ciò ha garantito l'estrazione delle informazioni per un totale di 8 file su 10, pertanto un rapporto che va oltre alla media, si osservi il risultato ottenuto in seguito al _run all_ in __json/metadata.json__.

Piccoli accorgimenti potrebbero essere utilizzati per i seguenti articoli, di cui non si ha il _DOI_:
- 91FeldmannMysliwietzMonien
- 07Beal
- 19Kamlish

Il file denominato _91FeldmannMysliwietzMonien_ presenta l'_identificativo digitale_ nella sezione __reference>unstructured__ della risposta ricevuta dalla _REST API_.
L'ultimo riportato è stato cercato manualmente all'interno dell'API di _CrossRef_; infatti non è stata delineata una determinata persistenza dei dati, anzi l'articolo scientifico non compare tra quelli proposti.